In [1]:
using LightGraphs

"""
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L = laplacian_matrix(g)
    a = lufact(L) \ (b - mean(b))    
    p = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end
;

In [2]:
"""
soft threshold
"""
soft(c,lambda) = sign(c).*max(abs(c)-lambda/2,0)

"""
computes gradient
"""
function gradient2(a,a_0,u,L,rho,b,y)
    grad = y-invLogit(a+a_0)+(u' * L)' + rho*L*(L*a-b)
    return grad
end;


"""
computes hessian
"""
function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return hess
end;




"""
newton raphson for a update
"""
function newton(y_i,a_0,L,rho,b,u)
    a = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - pinv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
        println(diff)
    end
    if(iters == MAX_ITER)
        print("max iter reached")
    end
    return a
end
;

In [3]:
"""
ADMM Function 
"""
function ADMM(A,L,t,t_0,new, rho, lambda)
	a = zeros(t-1,new)
	b = zeros(t-1)
	u = zeros(t-1,new)
#	alpha = 1.5  #relaxation parameter
	iters = 0
	diff = 1.0
	b_old = b
	while(diff >STOP_DIFF && iters< MAX_ITER )
        #a update and u update
        for i in 1:new
            a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
        end
        #b update
        c = zeros(t-1)
        for i in 1:numnewnodes
            c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/(new*rho)
        end
        b = soft(c[1:t_0],lambda)
        for i in 1:new
            u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
        end
        diff  = norm(b-b_old)
        b_old = b
	end
	return b
end
;

Current version

In [6]:
    for i in 1:new
        a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
    end
    #b update
    c = zeros(t-1)
    for i in 1:numnewnodes
        c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/(new*rho)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
        u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)

14.51563145609162


In [9]:
for j in 1:80     
    for i in 1:new
        a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
    end
    #b update
    c = zeros(t-1)
    for i in 1:numnewnodes
        c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/(new*rho)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
        u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end

0.4172109680314262
0.30450667678938714
0.20035854592400426
0.10153260776318973
0.007578639720435376
0.0819037452740119
0.16727014895547399
0.2488360058688457
0.32688065376950703
0.40165106827841157
0.47336518377140635
0.5422149971913086
0.6083691040881192
0.6719753269073081
0.7331628390367584
0.7920438521885984
0.8487165278300921
0.9032652118555546
0.9557624606415018
1.0062704780150824
1.0548422003549973
1.10152236804349
1.1463484983193482
92.67589956158324
3.885959947322678
4.056411246066192
4.385700699687362
4.792166389290743
5.405643790889636
5.270111376006137
5.378405681519967
5.194663320382762
5.33797297022308
87.70099662915788
7.6248893637736845
8.181358529257468
8.546939762962422
115.11101289143467
13.67519627903399
82.24306577099665
16.391726420976575
17.010042651392595
17.439382078522485
17.759788019656007
18.06165340382801
17.574351456897983
17.234225700988304
16.547645849858007
79.41395530152603
108.63751007255344
78.01801682218354
106.82417338093688
21.381817429045565
22.12

In [3]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*1. 
genb = copy(b)  # save for later
g = randEdgeGen(g,1000)
A = Array{Int64,2}[]
L =  SparseMatrixCSC{Int64,Int64}[]
push!(L, laplacian_matrix(g))
numnewnodes = 1
a_0 = -5
# creates matrix A and L where A[i] is the connections for ith node and L[i] is the laplacian of the i-1st time step 
for i in 1:numnewnodes  
    g = addPrefNode(g,b, a_0)
    push!(L,laplacian_matrix(g))
    connects = zeros(2^levels-2+i,1)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end


t = 2^levels-1+numnewnodes #number of nodes at time t
t_0 = 2^levels-1  # number of initial nodes
;

In [4]:
const MAX_ITER = 1000
const STOP_DIFF = 0.001;

rho = 0.1
lambda = 1.1
new = numnewnodes
a = zeros(t-1,new)
b = zeros(t_0)
u = zeros(t-1,new)
#	alpha = 1.5  #relaxation parameter
iters = 0
diff = 1.0
b_old = b;


Makes b length t_0, then appends 0's when passing to a and u update

In [ ]:
    for i in 1:new
        a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
        u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
    end
    #b update
    c = zeros(t-1)
    for i in 1:numnewnodes
        c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/((t-t_0)*rho/2)
    end
    b = soft(c[1:t_0],lambda)
    diff  = norm(b-b_old)
    b_old = b
    println(diff)

Switched dual update to end.NOTE diff = 1 after first iter

In [ ]:
    for i in 1:new
        a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
    end
    #b update
    c = zeros(t-1)
    for i in 1:numnewnodes
        c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/((t-t_0)*rho/2)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
        u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)

subtract mean from b

In [12]:
for j in 1:5    
    for i in 1:new
        a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
    end
    #b update
    c = zeros(t-1)
    for i in 1:numnewnodes
        c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/((t-t_0)*rho/2)
    end
    b = soft(c[1:t_0],lambda)
    b = b- mean(b)
    for i in 1:new
        u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end

25.208305390896683


LoadError: InterruptException:

CURRENT VERSION. Revised b update

In [8]:
for j in 1:200     
    for i in 1:new
        a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
    end
    #b update
    c = zeros(t-1)
    for i in 1:numnewnodes
        c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/(new*rho)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
        u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end

63.18697800915604
22.129660293905086
62.0297153793871
22.232803501924018
54.82119180926336
88.07210044348219
42.70537649437791
73.89411354460331
23.131949344350105
22.4277044170348
21.864265218403624
42.04285794520239
21.29428514392673
21.120007105089375
20.521598837804646
42.05250468302044
67.86234782129881
87.81950972464035
21.21437487621987
20.946437771982026
20.436670282770898
20.10700325865751
19.819446936972643
42.385361229588696
42.19578702344731
19.96158457085107
41.955045130456675
19.90790849269118
162.30672753936065
135.931541077637
61.70213980002035
76.08539318212152
23.423967612633298
43.26592875874465
21.541308765456392
49.294023263239744
66.27315570181474
20.51260686363439
20.309079648269268
20.082138974129872
124.55024536581126
21.225619489407556
67.15530418622744
69.45993268944966
41.80283566999477
22.87509431278141
66.74867379815457
23.061891062931075
41.758345762094756
66.26825853664613
41.77273780363409
21.424875717099777
169.50507274113622
54.98141703679023
141.6904

In [7]:
for k in 1:6
    rho = 0.05+0.05*k
    println(string("attempt ",k,"rho = ",rho))
    lambda = 1.1
    new = numnewnodes
    a = zeros(t-1,new)
    b = zeros(t_0)
    u = zeros(t-1,new)
    #	alpha = 1.5  #relaxation parameter
    iters = 0
    diff = 1.0
    b_old = b;
    for j in 1:50     
        for i in 1:new
            a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
        end
        #b update
        c = zeros(t-1)
        for i in 1:numnewnodes
            c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/(new*rho)
        end
        b = soft(c[1:t_0],lambda)
        for i in 1:new
            u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
        end
        diff  = norm(b-b_old)
        b_old = b
        println(diff)
    end
end

attempt 1rho = 0.1
4.179744059634811
3.4042073473227936
3.0646780711761807
2.779449145988138
2.649876299640597
10.61678122426692
3.3626399434976455
3.110433399125731
2.9800738030472376
2.784653695732113
2.482687766231453
9.467170513380845
2.4121977771320777
2.174379331115194
1.9173479784088772
1.7022781314049418
1.554270638873892
1.4195532619602882
9.171000694869136
1.436326844200225
1.290378153486952
1.2392070796635548
1.333101855269041
1.5075199008611644
1.5681659128135517
1.4933375857789009
1.2970677641488035
1.2187701888391305
1.189049190968547
11.885282170617092
13.252386963706728
1.9762536005584046
1.9248924763604855
1.792958205487352
1.6668873523821763
1.5723128642144886
1.517911649317086
1.4354025032112165
1.363620237957697
1.2910973031260218
1.2094537957794034
1.1521383966053274
1.1181335901300478
1.112961900887515
1.1441595037811834
1.20224060324882
1.2515118018296696
1.2602852879343724
1.2319715065895402
1.1914151629697882
attempt 2rho = 0.15000000000000002
2.271402945906848

In [8]:
string("attempt ",k)

"attempt1"

In [13]:
sum(invLogit(a[:,1]-5))

9.026771403697222

In [ ]:
sum(b)